In [1]:
import pandas as pd
import math
import csv
import random
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import cross_val_score

In [36]:
# 当每支队伍没有elo等级分时，赋予其基础elo等级分
base_elo = 1600
team_elos = {} 
team_stats = {}
X = []
y = []
# 存放数据的目录
folder = 'data' 

In [3]:
# 根据每支队伍的Miscellaneous Opponent，Team统计数据csv文件进行初始化
def initialize_data_MOT(Mstat, Ostat, Tstat):
    new_Mstat = Mstat.drop(['Rk', 'Arena'], axis=1)
    new_Ostat = Ostat.drop(['Rk', 'G', 'MP'], axis=1)
    new_Tstat = Tstat.drop(['Rk', 'G', 'MP'], axis=1)

    team_stats1 = pd.merge(new_Mstat, new_Ostat, how='left', on='Team')
    team_stats1 = pd.merge(team_stats1, new_Tstat, how='left', on='Team')
    return team_stats1.set_index('Team', inplace=False, drop=True)
def initialize_data_S(Sche):
    new_Sche = Sche.drop(['Date','Start','Attend.','Notes'],axis=1)   # already sort in game date
    new_Sche["HomeWin"] = new_Sche["PTSV"] < new_Sche["PTSH"]
    return new_Sche

In [5]:
# 获取每支队伍的`Elo Score`等级分函数，当在开始没有等级分时，将其赋予初始`base_elo`值：
def get_elo(team):
    try:
        return team_elos[team]
    except:
        # 当最初没有elo时，给每个队伍最初赋base_elo
        team_elos[team] = base_elo  #1600
        return team_elos[team]

In [6]:
# 定义计算每支球队的Elo等级分函数
# 计算每个球队的elo值
def calc_elo(win_team, lose_team):
    winner_rank = get_elo(win_team)
    loser_rank = get_elo(lose_team)
    rank_diff = winner_rank - loser_rank
    exp = (rank_diff  * -1) / 400
    odds = 1 / (1 + math.pow(10, exp))
    # 根据rank级别修改K值
    if winner_rank < 2100:
        k = 32
    elif winner_rank >= 2100 and winner_rank < 2400:
        k = 24
    else:
        k = 16
    # 更新 rank 数值
    new_winner_rank = round(winner_rank + (k * (1 - odds)))      
    new_loser_rank = round(loser_rank + (k * (0 - odds)))
    return new_winner_rank, new_loser_rank


In [26]:
#基于我们初始好的统计数据，及每支队伍的 **Elo score** 计算结果，建立对应 2016~2017 年常规赛和季后赛中每场比赛的数据集
# （在主客场比赛时，我们认为主场作战的队伍更加有优势一点，因此会给主场作战队伍相应加上 100 等级分）：
def  build_dataSet(all_data):
    print("Building data set..")
    X = []
    skip = 0
    for index, row in all_data.iterrows():
        if(row['HomeWin'] == True):
            Wteam = row['Home/Neutral']
            Lteam = row['Visitor/Neutral']
        else:
            Wteam = row['Visitor/Neutral']
            Lteam = row['Home/Neutral']
        
        #获取最初的elo或是每个队伍最初的elo值
        team1_elo = get_elo(Wteam)
        team2_elo = get_elo(Lteam)

        # 给主场比赛的队伍加上100的elo值
        (row['HomeWin'] == True)
        if (Wteam == row["Home/Neutral"]):
            team1_elo += 100
        else:
            team2_elo +100
#         if row['WLoc'] == 'H':
#             team1_elo += 100
#         else:
#             team2_elo += 100

        # 把elo当为评价每个队伍的第一个特征值
        team1_features = [team1_elo]
        team2_features = [team2_elo]

        # 添加我们从basketball reference.com获得的每个队伍的统计信息
        for key, value in team_stats.loc[Wteam].iteritems():
            team1_features.append(value)
        for key, value in team_stats.loc[Lteam].iteritems():
            team2_features.append(value)

        # 将两支队伍的特征值随机的分配在每场比赛数据的左右两侧
        # 并将对应的0/1赋给y值
        if random.random() > 0.5:
            X.append(team1_features + team2_features)  # 67+1.  +. 67+1
            y.append(0)
        else:
            X.append(team2_features + team1_features)
            y.append(1)

#         if skip == 0:
#             print('X',X)
#             skip = 1

        # 根据这场比赛的数据更新队伍的elo值
        new_winner_rank, new_loser_rank = calc_elo(Wteam, Lteam)
        team_elos[Wteam] = new_winner_rank
        team_elos[Lteam] = new_loser_rank
# 使用0代替数组x中的nan元素，使用有限的数字代替inf元素
    return np.nan_to_num(X), y



In [37]:
Mstat = pd.read_csv("16-17/16_17_Miscellaneous_Stats.csv",delimiter = ",")
Ostat = pd.read_csv("16-17/16_17_Opponent_Per_Game_Stats.csv",delimiter = ",")
Tstat = pd.read_csv("16-17/16_17_Team_Per_Game_Stats.csv",delimiter = ",")

team_stats = initialize_data_MOT(Mstat, Ostat, Tstat)
# print(type(team_stats))
print(team_stats.shape)
# print(team_stats.head())
# print(team_stats.loc["Atlanta Hawks"])
# print(type(team_stats.loc["Atlanta Hawks*"]))
result_data = pd.read_csv("16-17/16_17_Schedual.csv",delimiter = ",")
schedual_1617_stats = initialize_data_S(result_data)
print(schedual_1617_stats.shape)

X, y = build_dataSet(schedual_1617_stats)
print("*"*100)
print(X.shape)
print(len(y))
print("*"*100)
                              
    
#     # 训练网络模型
print("Fitting on %d game samples.." % len(X))
model = linear_model.LogisticRegression()
model.fit(X, y)

# 利用10折交叉验证计算训练正确率
print("Doing cross-validation..")
print(cross_val_score(model, X, y, cv = 10, scoring='accuracy', n_jobs=-1).mean())

(30, 67)
(1309, 5)
Building data set..
****************************************************************************************************
(1309, 136)
1309
****************************************************************************************************
Fitting on 1309 game samples..
Doing cross-validation..
0.710446271285966


In [53]:
# pytorch 
import torch
from torch.autograd import Variable as V
import torch.nn.functional as F

Mstat = pd.read_csv("16-17/16_17_Miscellaneous_Stats.csv",delimiter = ",")
Ostat = pd.read_csv("16-17/16_17_Opponent_Per_Game_Stats.csv",delimiter = ",")
Tstat = pd.read_csv("16-17/16_17_Team_Per_Game_Stats.csv",delimiter = ",")
team_stats = initialize_data_MOT(Mstat, Ostat, Tstat)
print(team_stats.shape)
result_data = pd.read_csv("16-17/16_17_Schedual.csv",delimiter = ",")
schedual_1617_stats = initialize_data_S(result_data)
print(schedual_1617_stats.shape)
X =[]
y =[]
X, y = build_dataSet(schedual_1617_stats)
print("*"*100)
print(X.shape)
print(len(y))
print("*"*100)
 
X_tensor = torch.tensor(X,dtype=torch.float)
y_tensor = torch.tensor(y,dtype=torch.float).view(-1,1)

print(X_tensor.shape)
print(y_tensor.shape)

class Model(torch.nn.Module): 
    def __init__(self):
        super(Model, self).__init__() 
        self.linear1 = torch.nn.Linear(136, 80) 
        self.linear2 = torch.nn.Linear(80, 1) 
        self.sigmoid = torch.nn.Sigmoid()  #和上节的nn.function下的不一样
#         self.activate = torch.nn.Relu()
    def forward(self, x):
        x = self.sigmoid(self.linear1(x)) 
        x = self.sigmoid(self.linear2(x)) 
        #就算是用relu最后一层输出前也得用sigmoid
        return x
model = Model()
criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

for epoch in range(10): 
    # Forward
    #注意这里没有minibatch，是整个batch
    y_pred = model(X_tensor)
    loss = criterion(y_pred, y_tensor)
    print(epoch, loss.item())
    # Backward
    optimizer.zero_grad()
    loss.backward()
    # Update
    optimizer.step()

(30, 67)
(1309, 5)
Building data set..
****************************************************************************************************
(1309, 136)
1309
****************************************************************************************************
torch.Size([1309, 136])
torch.Size([1309, 1])
0 0.6949395537376404
1 0.6944552063941956
2 0.691913902759552
3 0.6917850375175476
4 0.6916497349739075
5 0.6915233731269836
6 0.691398561000824
7 0.6912616491317749
8 0.691139280796051
9 0.6910221576690674
